In [1]:
import pandas as pd
import numpy as np

In [2]:
# Before you doing this, you need to remove the first line(header) of the below file
match_best = pd.read_csv('Algorithm/match_best.csv', header=None)
match_second = pd.read_csv('Algorithm/match_second.csv', header=None)
match_third = pd.read_csv('Algorithm/match_third.csv', header=None)
raw_data = pd.read_csv('Algorithm/raw_data.csv', header=None)

In [3]:
U = match_second - match_third

In [4]:
U

,0,1,2,3,4,5,6,7,8,9,...,104,105,106,107,108,109,110,111,112,113
0,0.089776,-0.007110,-0.111134,0.260840,0.379032,0.000000,0.000000,-0.008485,0.009398,0.002909,...,-0.010719,0.005834,-0.010719,-0.001628,-0.001628,-0.001628,0.005834,-0.001628,0.005834,0.005834
1,0.189703,-0.168859,0.295482,-0.298384,-0.131048,0.000000,-0.438433,-0.657982,-0.728075,-0.741854,...,-0.010692,0.005569,0.122007,-0.001281,0.012418,-0.009411,-0.001281,-0.106972,-0.075732,-0.001281
2,-0.029535,-0.061936,0.046793,-0.082618,-0.088710,-0.894737,-0.001866,-0.005544,-0.004934,-0.000323,...,-0.190000,-0.006667,0.021667,0.001667,0.001667,0.001667,0.011667,0.001667,-0.048333,-0.048333
3,0.130852,0.061352,-0.318677,-0.176755,0.274194,0.000000,-0.014925,-0.022619,-0.065420,-0.007870,...,-0.012248,0.005504,-0.002248,-0.002248,-0.002248,-0.002248,-0.012248,-0.012248,-0.002248,-0.132248
4,0.160522,0.055905,-0.012303,-0.092753,0.441532,0.000000,0.981343,0.435019,0.145198,0.612054,...,0.005316,-0.033425,-0.007490,-0.023115,-0.002497,-0.002497,-0.012806,-0.012806,0.005316,-0.002497
5,0.096459,-0.035718,0.173054,0.006380,0.338710,0.000000,0.041045,0.029308,0.079813,0.028379,...,-0.002220,-0.002220,-0.002220,-0.002220,-0.043887,-0.002220,0.005977,-0.002220,-0.002220,-0.002220
6,0.094456,-0.032427,0.347317,0.136893,-0.284274,0.827170,0.018657,0.015852,0.015394,0.003198,...,-0.017928,-0.000458,-0.275023,-0.009078,0.017013,0.189426,0.008392,-0.000229,0.008392,-0.000458
7,0.087431,0.011401,-0.128883,0.016864,-0.211694,0.020942,-0.009328,-0.036727,-0.063239,-0.012097,...,0.000072,-0.008402,0.000072,0.000072,0.017239,0.000072,0.000072,0.000072,0.017239,0.025713
8,0.145651,-0.008258,-0.301129,-0.237811,-0.100806,0.000000,0.027985,0.013899,-0.001911,0.001905,...,0.061034,-0.001513,-0.221816,-0.000504,0.005170,0.268726,-0.033291,0.014880,-0.001513,-0.000504
9,0.047587,-0.096851,0.432634,0.030610,0.114919,0.000000,-0.768657,-0.154663,0.075763,-0.419822,...,-0.000194,-0.000194,0.015679,-0.000194,-0.000194,0.015679,-0.008324,-0.016454,0.007743,-0.000194


In [5]:
# data = np.identity(65)  # without topic features
data = np.identity(115)   # with topic features

In [6]:
data

array([[ 1.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  1.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  1.]])

In [7]:
I = pd.DataFrame(data=data[1:,1:])

In [8]:
I

,0,1,2,3,4,5,6,7,8,9,...,104,105,106,107,108,109,110,111,112,113
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
9,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [9]:
parameter_l = 0.5
parameter_u = 0.5
C = pd.concat([parameter_l * I, parameter_u * U],ignore_index=True)

In [10]:
C

,0,1,2,3,4,5,6,7,8,9,...,104,105,106,107,108,109,110,111,112,113
0,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [13]:
def S(x) :
    if x > -1 and x < 1:
        return x
    elif x >= 1 :
        return 1
    else :
        return -1

# test the S(x)
print S(0)
print S(0.5)
print S(-0.7)
print S(-2)
print S(1.8)

0
0.5
-0.7
-1
1


In [15]:
random_num = np.random.uniform(-1, 1, size=114)

In [16]:
w = pd.DataFrame(data=random_num)

In [17]:
w

,0
0,0.787341
1,-0.272060
2,-0.843828
3,-0.900000
4,0.735545
5,-0.583294
6,0.221035
7,-0.949888
8,-0.239338
9,0.459914


In [18]:
parameter_g = 0.5
C_Dot_W = C.dot(w) / parameter_g

In [19]:
C_Dot_W

,0
0,0.787341
1,-0.272060
2,-0.843828
3,-0.900000
4,0.735545
5,-0.583294
6,0.221035
7,-0.949888
8,-0.239338
9,0.459914


In [20]:
A = C_Dot_W[0].apply(S)

In [21]:
A

0       0.787341
1      -0.272060
2      -0.843828
3      -0.900000
4       0.735545
5      -0.583294
6       0.221035
7      -0.949888
8      -0.239338
9       0.459914
10     -0.507381
11      0.220190
12      0.728194
13     -0.703629
14     -0.304613
15     -0.056550
16     -0.388806
17      0.570085
18     -0.883635
19      0.138712
20     -0.210437
21      0.996339
22     -0.944380
23      0.078776
24      0.269562
25     -0.217593
26     -0.959748
27      0.789158
28     -0.585234
29     -0.531367
          ...   
3424    0.667590
3425   -0.136000
3426    0.562560
3427   -0.763560
3428    0.122679
3429    0.220663
3430   -0.142226
3431   -0.766768
3432   -0.019374
3433    0.356490
3434    0.601297
3435    0.318101
3436    0.269439
3437   -0.071731
3438   -0.208745
3439    0.757792
3440    0.821870
3441   -0.493363
3442    0.391910
3443   -0.292237
3444   -0.135490
3445    0.036326
3446    0.231802
3447    0.477433
3448   -0.150128
3449    0.041298
3450   -0.594068
3451    1.0000

In [22]:
# C_T : C 's transpose
C_T = C.transpose()

In [23]:
C_T_Dot_A = 0.5 * C_T.dot(A)

In [24]:
C_T_Dot_A

0        3.760198
1       -4.046811
2      -33.413623
3       -1.931760
4        8.454827
5      -34.534623
6        0.633122
7       -4.561123
8       -1.705289
9        1.069823
10       1.436127
11       9.943935
12       4.456273
13       2.814947
14     -15.468635
15      41.563477
16     -72.766330
17      77.183826
18      25.301585
19      47.777310
20      53.046163
21     104.385818
22      32.701211
23     -88.585390
24     -19.870584
25      32.471476
26     -79.528040
27      94.606443
28     -49.549667
29     -63.345669
          ...    
84      -0.878424
85       0.984864
86      -3.023917
87      -2.718330
88       2.309615
89       8.296232
90      -0.389516
91       3.609768
92       3.269914
93      -2.233451
94       3.480785
95       0.535804
96     -10.265773
97       1.066283
98      17.749508
99      -1.512272
100    -16.060621
101      1.214751
102      1.719983
103      2.801054
104     -3.298379
105     -0.370622
106      5.219447
107      4.722765
108     -0

In [25]:
raw_data

,0,1,2,3,4,5,6,7,8,9,...,105,106,107,108,109,110,111,112,113,114
0,0.186485,0.004280,-0.052365,-0.100313,-0.119355,0.000000,0.026119,0.042349,0.015746,0.002408,...,-0.000395,-0.009486,-0.000395,-0.000395,-0.000395,0.173518,0.051779,-0.000395,-0.000395,1
1,0.112752,-0.282833,0.118243,-0.296612,0.095027,0.000000,-0.902985,-0.982587,-0.989291,-0.996494,...,0.007374,-0.000378,0.030630,-0.000378,-0.008508,-0.000378,-0.098317,-0.081679,-0.000378,1
2,0.164212,-0.104463,0.165963,-0.059522,0.640457,0.000000,0.014925,0.016112,0.011910,0.003110,...,-0.010338,0.016983,-0.002004,-0.002004,0.016983,0.162553,-0.002004,-0.052004,-0.045675,1
3,0.210626,-0.007786,0.074747,-0.145550,-0.471237,0.000000,-0.018657,-0.022567,-0.065482,-0.008832,...,-0.000099,-0.000099,-0.000099,-0.000099,0.029604,-0.010099,0.009703,-0.000099,-0.090495,1
4,0.139473,-0.046464,0.032939,0.186306,-0.006788,-0.015707,0.186567,0.134881,0.174565,0.173687,...,-0.031891,-0.021582,0.184025,-0.000963,-0.000963,-0.011273,-0.001927,-0.000963,-0.000963,1
5,0.172734,-0.073209,0.158361,-0.011459,-0.108199,0.000000,-0.001866,-0.019287,0.000304,-0.001710,...,0.025619,-0.001408,0.007601,-0.043074,-0.001408,-0.001408,-0.001408,-0.001408,-0.001408,1
6,0.180033,-0.071145,0.066301,-0.000798,-0.520161,0.837696,0.037313,0.020224,0.005982,0.003552,...,0.010079,-0.291626,-0.008440,0.000410,0.009669,0.000410,0.018928,0.000410,-0.008440,1
7,0.113538,-0.041266,-0.197213,0.016080,-0.202151,0.020942,-0.035448,-0.045108,-0.071002,-0.012644,...,0.053226,0.000297,0.000297,-0.008177,0.000297,0.000297,0.009069,-0.008177,0.000297,1
8,0.183023,0.000301,-0.407939,-0.186576,0.059610,-0.068063,-0.009328,-0.008355,-0.010881,-0.001708,...,-0.001863,-0.206781,-0.000621,-0.033408,0.014531,-0.033408,-0.000621,-0.017014,0.014531,1
9,0.076459,-0.123809,0.153716,0.094672,-0.229570,0.000000,-0.951493,-0.388193,-0.105502,-0.594194,...,0.000719,0.000719,0.009569,0.000719,0.000719,0.010289,-0.015541,0.000719,0.000719,1


In [26]:
# without topic features
#X = raw_data.ix[0:,0:63]
#Y = raw_data.ix[0:,64]

# with topic features
X = raw_data.ix[0:, 0:113]
Y = raw_data.ix[0:, 114]

In [27]:
X

,0,1,2,3,4,5,6,7,8,9,...,104,105,106,107,108,109,110,111,112,113
0,0.186485,0.004280,-0.052365,-0.100313,-0.119355,0.000000,0.026119,0.042349,0.015746,0.002408,...,-0.009486,-0.000395,-0.009486,-0.000395,-0.000395,-0.000395,0.173518,0.051779,-0.000395,-0.000395
1,0.112752,-0.282833,0.118243,-0.296612,0.095027,0.000000,-0.902985,-0.982587,-0.989291,-0.996494,...,-0.016638,0.007374,-0.000378,0.030630,-0.000378,-0.008508,-0.000378,-0.098317,-0.081679,-0.000378
2,0.164212,-0.104463,0.165963,-0.059522,0.640457,0.000000,0.014925,0.016112,0.011910,0.003110,...,-0.181013,-0.010338,0.016983,-0.002004,-0.002004,0.016983,0.162553,-0.002004,-0.052004,-0.045675
3,0.210626,-0.007786,0.074747,-0.145550,-0.471237,0.000000,-0.018657,-0.022567,-0.065482,-0.008832,...,-0.010099,-0.000099,-0.000099,-0.000099,-0.000099,0.029604,-0.010099,0.009703,-0.000099,-0.090495
4,0.139473,-0.046464,0.032939,0.186306,-0.006788,-0.015707,0.186567,0.134881,0.174565,0.173687,...,-0.000963,-0.031891,-0.021582,0.184025,-0.000963,-0.000963,-0.011273,-0.001927,-0.000963,-0.000963
5,0.172734,-0.073209,0.158361,-0.011459,-0.108199,0.000000,-0.001866,-0.019287,0.000304,-0.001710,...,0.214809,0.025619,-0.001408,0.007601,-0.043074,-0.001408,-0.001408,-0.001408,-0.001408,-0.001408
6,0.180033,-0.071145,0.066301,-0.000798,-0.520161,0.837696,0.037313,0.020224,0.005982,0.003552,...,-0.017289,0.010079,-0.291626,-0.008440,0.000410,0.009669,0.000410,0.018928,0.000410,-0.008440
7,0.113538,-0.041266,-0.197213,0.016080,-0.202151,0.020942,-0.035448,-0.045108,-0.071002,-0.012644,...,0.000297,0.053226,0.000297,0.000297,-0.008177,0.000297,0.000297,0.009069,-0.008177,0.000297
8,0.183023,0.000301,-0.407939,-0.186576,0.059610,-0.068063,-0.009328,-0.008355,-0.010881,-0.001708,...,-0.000621,-0.001863,-0.206781,-0.000621,-0.033408,0.014531,-0.033408,-0.000621,-0.017014,0.014531
9,0.076459,-0.123809,0.153716,0.094672,-0.229570,0.000000,-0.951493,-0.388193,-0.105502,-0.594194,...,0.000719,0.000719,0.000719,0.009569,0.000719,0.000719,0.010289,-0.015541,0.000719,0.000719


In [28]:
Y 

0        1
1        1
2        1
3        1
4        1
5        1
6        1
7        1
8        1
9        1
10       1
11       1
12       1
13       1
14       1
15       1
16       1
17       1
18       1
19       1
20       1
21       1
22       1
23       1
24       1
25       1
26       1
27       1
28       1
29       1
        ..
33370   -1
33371   -1
33372   -1
33373   -1
33374   -1
33375   -1
33376   -1
33377   -1
33378   -1
33379   -1
33380   -1
33381   -1
33382   -1
33383   -1
33384   -1
33385   -1
33386   -1
33387   -1
33388   -1
33389   -1
33390   -1
33391   -1
33392   -1
33393   -1
33394   -1
33395   -1
33396   -1
33397   -1
33398   -1
33399   -1
Name: 114, dtype: int64

In [29]:
X_Mul_Y = X.mul(Y, axis=0)

In [30]:
X_Mul_Y

,0,1,2,3,4,5,6,7,8,9,...,104,105,106,107,108,109,110,111,112,113
0,0.186485,0.004280,-0.052365,-0.100313,-0.119355,0.000000,0.026119,0.042349,0.015746,0.002408,...,-0.009486,-0.000395,-0.009486,-0.000395,-0.000395,-0.000395,0.173518,0.051779,-0.000395,-0.000395
1,0.112752,-0.282833,0.118243,-0.296612,0.095027,0.000000,-0.902985,-0.982587,-0.989291,-0.996494,...,-0.016638,0.007374,-0.000378,0.030630,-0.000378,-0.008508,-0.000378,-0.098317,-0.081679,-0.000378
2,0.164212,-0.104463,0.165963,-0.059522,0.640457,0.000000,0.014925,0.016112,0.011910,0.003110,...,-0.181013,-0.010338,0.016983,-0.002004,-0.002004,0.016983,0.162553,-0.002004,-0.052004,-0.045675
3,0.210626,-0.007786,0.074747,-0.145550,-0.471237,0.000000,-0.018657,-0.022567,-0.065482,-0.008832,...,-0.010099,-0.000099,-0.000099,-0.000099,-0.000099,0.029604,-0.010099,0.009703,-0.000099,-0.090495
4,0.139473,-0.046464,0.032939,0.186306,-0.006788,-0.015707,0.186567,0.134881,0.174565,0.173687,...,-0.000963,-0.031891,-0.021582,0.184025,-0.000963,-0.000963,-0.011273,-0.001927,-0.000963,-0.000963
5,0.172734,-0.073209,0.158361,-0.011459,-0.108199,0.000000,-0.001866,-0.019287,0.000304,-0.001710,...,0.214809,0.025619,-0.001408,0.007601,-0.043074,-0.001408,-0.001408,-0.001408,-0.001408,-0.001408
6,0.180033,-0.071145,0.066301,-0.000798,-0.520161,0.837696,0.037313,0.020224,0.005982,0.003552,...,-0.017289,0.010079,-0.291626,-0.008440,0.000410,0.009669,0.000410,0.018928,0.000410,-0.008440
7,0.113538,-0.041266,-0.197213,0.016080,-0.202151,0.020942,-0.035448,-0.045108,-0.071002,-0.012644,...,0.000297,0.053226,0.000297,0.000297,-0.008177,0.000297,0.000297,0.009069,-0.008177,0.000297
8,0.183023,0.000301,-0.407939,-0.186576,0.059610,-0.068063,-0.009328,-0.008355,-0.010881,-0.001708,...,-0.000621,-0.001863,-0.206781,-0.000621,-0.033408,0.014531,-0.033408,-0.000621,-0.017014,0.014531
9,0.076459,-0.123809,0.153716,0.094672,-0.229570,0.000000,-0.951493,-0.388193,-0.105502,-0.594194,...,0.000719,0.000719,0.000719,0.009569,0.000719,0.000719,0.010289,-0.015541,0.000719,0.000719


In [31]:
Sum_X_Mul_Y = X_Mul_Y.sum()

In [32]:
Sum_X_Mul_Y

0       475.563529
1      -376.786845
2       113.336678
3       113.852853
4       150.296102
5       197.012786
6       131.727612
7       121.765090
8       110.762348
9        86.431075
10     1022.256754
11     4041.581702
12      -33.666667
13     1296.000000
14    -1226.644937
15      417.430723
16     -157.438034
17      706.426992
18     1686.171392
19      -66.714249
20      839.716462
21      -15.209439
22     -241.487967
23       76.380264
24      352.576316
25     2208.105731
26      856.891584
27      721.843235
28     -455.674659
29    -1191.061070
          ...     
84      -54.505931
85        1.832480
86       99.561784
87        8.187841
88      -11.477862
89       75.306861
90        5.198588
91      -39.069478
92       43.455974
93      -37.212931
94      151.649839
95        7.900578
96       21.396581
97      -12.484413
98       38.816809
99       44.754627
100      83.098873
101      45.383817
102     -33.925922
103      35.029103
104     168.361144
105     -38.

In [33]:
Target = Sum_X_Mul_Y - C_T_Dot_A

In [34]:
Target = Target.as_matrix()

In [35]:
Target

array([  4.71803331e+02,  -3.72740033e+02,   1.46750301e+02,
         1.15784613e+02,   1.41841275e+02,   2.31547409e+02,
         1.31094490e+02,   1.26326213e+02,   1.12467637e+02,
         8.53612521e+01,   1.02082063e+03,   4.03163777e+03,
        -3.81229395e+01,   1.29318505e+03,  -1.21117630e+03,
         3.75867246e+02,  -8.46717039e+01,   6.29243166e+02,
         1.66086981e+03,  -1.14491559e+02,   7.86670299e+02,
        -1.19595257e+02,  -2.74189178e+02,   1.64965654e+02,
         3.72446900e+02,   2.17563426e+03,   9.36419624e+02,
         6.27236792e+02,  -4.06124992e+02,  -1.12771540e+03,
        -1.24949767e+03,  -1.08948742e+03,  -4.29808056e+02,
        -3.08117003e+02,   2.38055169e+03,  -9.66572239e+02,
        -1.29932752e+02,   6.15547209e+02,  -1.42584729e+02,
         6.18740953e+02,   8.28211205e+02,   7.45601593e+01,
        -1.68298703e+02,  -8.03080342e+02,   6.54999527e+02,
        -8.90193615e+02,  -3.28752175e+02,  -1.29155719e+03,
         1.03433167e+03,

In [36]:
X_T = X.transpose()

In [37]:
X_T_Dot_X = X_T.dot(X)

In [38]:
X_T_Dot_X

,0,1,2,3,4,5,6,7,8,9,...,104,105,106,107,108,109,110,111,112,113
0,527.396844,-69.311908,7.717840,39.981302,96.515592,48.788245,16.825563,23.481714,19.872939,4.673424,...,12.087575,1.050919,3.087724,-4.518545,1.864577,2.375483,3.940575,-1.896422,-0.003414,-1.653950
1,-69.311908,372.969128,-21.995324,71.921884,215.012894,142.486052,84.773697,93.123594,79.745847,90.108614,...,-8.385408,-6.025489,2.086489,6.715379,1.492946,-2.241712,-1.630697,4.773955,4.119774,-2.262770
2,7.717840,-21.995324,2649.071851,-7.676603,129.938700,576.906649,172.011881,228.453374,172.040801,57.967910,...,39.286101,-2.440038,-7.598403,2.588151,-10.048967,2.531412,-2.583294,-12.053030,-14.926669,2.093940
3,39.981302,71.921884,-7.676603,696.693518,320.705100,205.650148,136.130985,172.173243,176.769716,178.978339,...,2.561652,2.348653,3.449564,1.035257,9.201645,-7.001041,-4.685534,1.369751,9.270129,-1.889940
4,96.515592,215.012894,129.938700,320.705100,4004.293870,1111.862786,467.194281,529.785949,400.306364,397.363426,...,15.992523,-17.952367,8.355076,-4.644302,-5.628741,0.405860,-9.502623,0.640297,19.896706,4.105179
5,48.788245,142.486052,576.906649,205.650148,1111.862786,4359.770134,427.186470,448.143739,398.238070,276.810004,...,12.111723,-2.987964,-0.836493,3.168467,-2.421870,-5.996132,4.456760,3.848223,2.742518,0.442614
6,16.825563,84.773697,172.011881,136.130985,467.194281,427.186470,1393.556109,1115.417140,902.413752,1066.661230,...,-10.773024,-5.832233,-7.593301,-11.410386,22.691592,-5.135078,-7.771093,5.475154,26.855695,3.027880
7,23.481714,93.123594,228.453374,172.173243,529.785949,448.143739,1115.417140,1190.213663,931.092319,961.235840,...,-6.589084,-6.249406,-8.470144,-10.323722,26.761270,-5.680612,-5.593591,6.952426,30.055435,0.737080
8,19.872939,79.745847,172.040801,176.769716,400.306364,398.238070,902.413752,931.092319,927.772981,853.643373,...,-0.151198,-5.999489,-4.479376,-7.078204,25.944948,-6.519346,-6.486177,3.458564,31.133756,-0.605936
9,4.673424,90.108614,57.967910,178.978339,397.363426,276.810004,1066.661230,961.235840,853.643373,998.122494,...,-8.748767,-5.622962,-6.503000,-8.511371,28.225027,-6.305105,-7.906501,4.584793,31.937961,-0.695267


In [39]:
Parameter = X_T_Dot_X.as_matrix()

In [40]:
w = np.linalg.solve(Parameter, Target)

In [41]:
w

array([ -4.35898980e+00,  -1.04955162e+00,   1.01865343e-01,
         1.92602715e+00,  -1.73797673e+00,   1.68077999e-02,
         7.97895111e-01,   1.25117202e+00,  -1.42994372e+00,
         7.35283683e-01,  -3.57227862e-01,   2.59128776e+00,
         1.04801000e+00,   1.03536314e+00,  -1.16629470e-01,
         1.83968694e-01,  -1.00431057e-01,   6.30144794e-01,
         8.25775801e-01,  -2.08810839e-01,   3.66037801e-01,
         1.08561540e+00,   7.91579751e-02,   2.76200000e-01,
        -1.76841930e-01,   4.36752676e-01,  -8.31087777e-02,
         1.72907502e-01,  -3.81567551e-01,  -9.06211732e-01,
         1.48964523e-01,   1.02838724e-02,  -3.78059541e-01,
         1.45514253e+00,   9.07767249e-02,  -4.00192191e-01,
         3.86011208e-01,   3.73387610e-01,   8.56910225e-01,
         4.60381626e-02,  -4.32644894e-01,  -4.70683751e-01,
         7.78026246e-01,  -3.98196311e-01,  -4.34086637e-01,
        -8.43467745e-02,   7.52313327e-01,   3.51461874e-01,
        -1.45196002e-01,

# Optimaization Algorithm

In [42]:
iterations = 0
delta_A = 1000
delta_w = 1000
bound = 0.0001

while delta_A > bound and delta_w > bound :
    iterations += 1
    
    C_Dot_W = C.dot(w) / parameter_g
    
    # update vector A
    A_now = C_Dot_W.apply(S)
    
    C_T_Dot_A = 0.5 * C_T.dot(A_now)
    Target = Sum_X_Mul_Y - C_T_Dot_A
    Target = Target.as_matrix()
    
    # update vector w
    w_now = np.linalg.solve(Parameter, Target)
    
    delta_A = np.linalg.norm(A_now-A)
    delta_w = np.linalg.norm(w_now-w)
    
    A = A_now
    w = w_now
    
    print 'iterations:' + str(iterations) + ' ' + \
            'delta_A:' + str(delta_A) + ' ' + \
            'delta_w:' + str(delta_w)  
print w

iterations:1 delta_A:75.8488462012 delta_w:1.27333949906e+15
iterations:2 delta_A:115.698802585 delta_w:2.26173709973e+15
iterations:3 delta_A:117.090844431 delta_w:2.26173709973e+15
iterations:4 delta_A:117.091914991 delta_w:2.26173709973e+15
iterations:5 delta_A:117.091393505 delta_w:2.26173709973e+15
iterations:6 delta_A:117.090693634 delta_w:2.26173709973e+15
iterations:7 delta_A:117.09079447 delta_w:2.26173709973e+15
iterations:8 delta_A:117.090563602 delta_w:2.26173709973e+15
iterations:9 delta_A:117.090962093 delta_w:2.26173709973e+15
iterations:10 delta_A:117.091593061 delta_w:2.26173709973e+15
iterations:11 delta_A:117.092036973 delta_w:2.26173709973e+15
iterations:12 delta_A:117.091404746 delta_w:2.26173709973e+15
iterations:13 delta_A:117.09072039 delta_w:2.26173709973e+15
iterations:14 delta_A:117.091177854 delta_w:2.26173709973e+15
iterations:15 delta_A:117.091043061 delta_w:2.26173709973e+15
iterations:16 delta_A:117.090582765 delta_w:2.26173709973e+15
iterations:17 delta

KeyboardInterrupt: 